<a href="https://colab.research.google.com/github/Edvandro-Nogueira/TCC-Univesp/blob/main/TCC_Modelo_fakeDETECTOR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Importações
import pandas as pd
from sklearn.model_selection import train_test_split
#from sklearn.feature_extraction.text import TfidfVectorizer
#from sklearn.svm import SVC
import pickle
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
import re
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
#from keras.preprocessing.sequence import pad_sequences

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
#Versão do python
import sys
print(sys.version)

3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]


In [4]:
dados_noticias = pd.read_table('https://raw.githubusercontent.com/Edvandro-Nogueira/TCC-Univesp/main/dados_noticias.csv', sep=";")
dados_noticias

,noticias,classe
0,katia abreu diz vai colocar expulsao moldura n...,0
1,ray peita bolsonaro conservador fake entrevist...,0
2,reinaldo azevedo desmascarado policia federal ...,0
3,relatorio assustador bndes mostra dinheiro pub...,0
4,radialista americano fala sobre pt vendem ilus...,0
...,...,...
10753,"Aug 26, 2020 — É falso que neurocientista brit...",0
10754,19/08/2020 - 17:45 / Atualizado em 19/08/2020 ...,0
10755,"Jun 6, 2020 — No Reino Unido, 80% da população...",0
10756,Estudo do pesquisador Karl Friston conclui que...,0


In [5]:
#Certificamos que existem apenas 2 objetos
dados_noticias['classe'].unique()

array([0, 1])

In [6]:
#Contando quantos falsos e quantos verdadeiros o dataset possui
contagem = dados_noticias['classe'].value_counts()
print(contagem)

0    6143
1    4615
Name: classe, dtype: int64


###Separando os dados em treino, validação e teste.

In [7]:
X = dados_noticias["noticias"]
y = dados_noticias["classe"]

# Etapa 1: Separar 90%-10% para teste final
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.1, random_state=42, stratify=y)

# Etapa 2: Separar 90% restantes em 80%-20% para treinamento e validação
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=42, stratify=y_train_val)

# Os dados de treinamento estão em X_train e y_train
# Os dados de validação estão em X_val e y_val
# Os dados de teste final estão em X_test e y_test

###Criando o modelo novo

In [ ]:
'''#Colocar stop words

# Fazendo a tokenização dos textos
tokenized_textos = [word_tokenize(str(texto)) for texto in dados_noticias["noticias"]]

# Convertendo as classes para rótulos numéricos
classes_numericas = []
classes_unicas = list(set(dados_noticias["classe"]))
for classe in dados_noticias["classe"]:
    classes_numericas.append(classes_unicas.index(classe))

# Tokenização dos textos de treinamento, validação e teste
tokenized_train = [word_tokenize(str(texto)) for texto in X_train]
tokenized_val = [word_tokenize(str(texto)) for texto in X_val]
tokenized_test = [word_tokenize(str(texto)) for texto in X_test]

# Vetorização dos textos usando o TF-IDF
vectorizer = TfidfVectorizer()
X_train_vectorized = vectorizer.fit_transform([' '.join(texto) for texto in tokenized_train])
X_val_vectorized = vectorizer.transform([' '.join(texto) for texto in tokenized_val])
X_test_vectorized = vectorizer.transform([' '.join(texto) for texto in tokenized_test])

# Criação e treinamento do modelo
model = SVC()
model.fit(X_train_vectorized, y_train)

# Avaliação do modelo
accuracy = model.score(X_val_vectorized, y_val)
print("Acurácia na validação: {:.2f}%".format(accuracy * 100))

# Avaliação do modelo no teste final
accuracy_test = model.score(X_test_vectorized, y_test)
print("Acurácia no teste final: {:.2f}%".format(accuracy_test * 100))'''

In [ ]:
'''# Previsões no conjunto de validação
y_pred_proba = model.decision_function(X_val_vectorized)
fpr, tpr, thresholds = roc_curve(y_val, y_pred_proba)
roc_auc = auc(fpr, tpr)

# Plot da curva ROC
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', label='ROC curve (AUC = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='red', linestyle='--', label='Random Guessing')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()'''

In [ ]:
'''# Criar um tokenizador simples
tokenizador = nltk.tokenize.TreebankWordTokenizer()

# Salvar o tokenizador
with open('tokenizador.pickle', 'wb') as handle:
    pickle.dump(tokenizador, handle, protocol=pickle.HIGHEST_PROTOCOL)'''

In [ ]:
'''# Carregando o tokenizador treinado anteriormente
with open('tokenizador.pickle', 'rb') as handle:
    tokenizador = pickle.load(handle)

# Nova entrada a ser avaliada
nova_entrada = "Oct 28, 2018 — ... em 631 das 645 cidades do estado de São Paulo no 2º turno das eleições de 2018. Fernando Haddad (PT) ganhou em apenas 14 municípios. ... Sandovalina e Euclides da Cunha Paulista), assim como no 1º turno. ... No geral, no estado, Bolsonaro teve 67,97% dos votos válidos e Haddad, 32,03%.	"

# Tokenize a nova entrada usando o tokenizador treinado
tokens = tokenizador.tokenize(nova_entrada)

# Vetorização dos tokens usando o TF-IDF
texto_vectorized = vectorizer.transform([' '.join(tokens)])

# Fazendo a previsão com o modelo treinado
previsao = model.predict(texto_vectorized)

# Convertendo o rótulo numérico da previsão para a classe original
classe_prevista = classes_unicas[previsao[0]]

# Imprimindo a classe prevista
print("Classe prevista:", classe_prevista)'''

###Modelo CNN

In [ ]:
'''label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(dados_noticias['classe'])'''

In [ ]:
'''train_texts, test_texts, train_labels, test_labels = train_test_split(dados_noticias['noticias'], labels, test_size=0.2, random_state=42)
train_texts = train_texts.astype(str)
test_texts = test_texts.astype(str)'''

In [ ]:
'''max_sequence_length = 1000

tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_texts)

train_sequences = tokenizer.texts_to_sequences(train_texts)
train_sequences = pad_sequences(train_sequences, maxlen=max_sequence_length)

test_sequences = tokenizer.texts_to_sequences(test_texts)
test_sequences = pad_sequences(test_sequences, maxlen=max_sequence_length)'''

In [ ]:
'''vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 100

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_sequence_length))
model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])'''

In [ ]:
'''model.fit(train_sequences, train_labels, epochs=3, batch_size=32, validation_data=(test_sequences, test_labels))

loss, accuracy = model.evaluate(test_sequences, test_labels)
print("Accuracy: {:.2f}%".format(accuracy * 100))'''

In [ ]:
'''#Fazendo previsão simples
text = "O aparecimento de surtos de Covid-19 em diferentes coros levantou a"
text = text.lower()
text = re.sub(r"[^\w\s]", "", text)
text_sequence = tokenizer.texts_to_sequences([text])
text_sequence = pad_sequences(text_sequence, maxlen=max_sequence_length)
prediction = model.predict(text_sequence)
#prediction = model.predict_classes(text_sequence)
prediction_classes = (prediction > 0.5).astype(int)
print(prediction_classes[0][0])'''

In [ ]:
#Segunda versão

In [10]:
train_texts = X_train.astype(str)
val_texts = X_val.astype(str)
test_texts = X_test.astype(str)

max_sequence_length = 1000

tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_texts)

train_sequences = tokenizer.texts_to_sequences(train_texts)
train_sequences = pad_sequences(train_sequences, maxlen=max_sequence_length)

val_sequences = tokenizer.texts_to_sequences(val_texts)
val_sequences = pad_sequences(val_sequences, maxlen=max_sequence_length)

test_sequences = tokenizer.texts_to_sequences(test_texts)
test_sequences = pad_sequences(test_sequences, maxlen=max_sequence_length)

vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 200

fakeDETECTOR = Sequential()
fakeDETECTOR.add(Embedding(vocab_size, embedding_dim, input_length=max_sequence_length))
fakeDETECTOR.add(Conv1D(256, 5, activation='relu'))
fakeDETECTOR.add(GlobalMaxPooling1D())
fakeDETECTOR.add(Dense(256, activation='relu'))
#fakeDETECTOR.add(Dense(128, activation='relu'))
fakeDETECTOR.add(Dense(1, activation='sigmoid'))

fakeDETECTOR.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

fakeDETECTOR.fit(train_sequences, y_train, epochs=4, batch_size=32, validation_data=(val_sequences, y_val))

loss, accuracy = fakeDETECTOR.evaluate(val_sequences, y_val)
print("Acurácia: {:.2f}%".format(accuracy * 100))

Epoch 1/4
243/243 [==============================] - 332s 1s/step - loss: 0.3568 - accuracy: 0.8462 - val_loss: 0.2524 - val_accuracy: 0.8916
Epoch 2/4
243/243 [==============================] - 329s 1s/step - loss: 0.0867 - accuracy: 0.9699 - val_loss: 0.2385 - val_accuracy: 0.9102
Epoch 3/4
243/243 [==============================] - 329s 1s/step - loss: 0.0111 - accuracy: 0.9966 - val_loss: 0.2851 - val_accuracy: 0.9128
Epoch 4/4
61/61 [==============================] - 18s 296ms/step - loss: 0.3032 - accuracy: 0.9112
Acurácia: 91.12%


In [ ]:
Mais camadas Densas não melhorou este modelo
Aumentar o numero de kernel não melhorou este modelo
Numero de epocas igual a 10 não melhorou este modelo, em todos os testes 3 epocas estão com uma acuracia melhor na validação e perca menor
Testando aumentar o numero de unidades --> Após 512 unidades o treinamento fica muito lento e não houve melhora no modelo.
Testando aumentar ou diminuir o numero de treinamento por lote
Testando mudar o otimizador


# 243/243 [==============================] - 164s 677ms/step - loss: 0.0171 - accuracy: 0.9944 - val_loss: 0.2746 - val_accuracy: 0.9107
fakeDETECTOR = Sequential()
fakeDETECTOR.add(Embedding(vocab_size, embedding_dim, input_length=max_sequence_length))
fakeDETECTOR.add(Conv1D(256, 5, activation='relu'))
fakeDETECTOR.add(GlobalMaxPooling1D())
fakeDETECTOR.add(Dense(256, activation='relu'))
#fakeDETECTOR.add(Dense(128, activation='relu'))
fakeDETECTOR.add(Dense(1, activation='sigmoid'))
fakeDETECTOR.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
fakeDETECTOR.fit(train_sequences, y_train, epochs=3, batch_size=32, validation_data=(val_sequences, y_val))

#243/243 [==============================] - 162s 667ms/step - loss: 0.0065 - accuracy: 0.9973 - val_loss: 0.3195 - val_accuracy: 0.9128
fakeDETECTOR = Sequential()
fakeDETECTOR.add(Embedding(vocab_size, embedding_dim, input_length=max_sequence_length))
fakeDETECTOR.add(Conv1D(256, 5, activation='relu'))
fakeDETECTOR.add(GlobalMaxPooling1D())
fakeDETECTOR.add(Dense(256, activation='relu'))
#fakeDETECTOR.add(Dense(128, activation='relu'))
fakeDETECTOR.add(Dense(1, activation='sigmoid'))
fakeDETECTOR.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
fakeDETECTOR.fit(train_sequences, y_train, epochs=4, batch_size=32, validation_data=(val_sequences, y_val))

In [18]:
#Fazendo previsão simples
text = "O aparecimento de surtos de Covid-19 em diferentes coros levantou a"
#text = "Infelizmente é confirmado ‘acaba de falecer’. Apresentador José Luiz Datena (com letras pequenas) sofre perda e está de (com letras grandes) Luto. LUT0: Infelizmente Nosso amado Datena da Band, horas após cirurgia delicada recebe pior das notícias"
text = text.lower()
text = re.sub(r"[^\w\s]", "", text)
text_sequence = tokenizer.texts_to_sequences([text])
text_sequence = pad_sequences(text_sequence, maxlen=max_sequence_length)
prediction = fakeDETECTOR.predict(text_sequence)
#prediction = fakeDETECTOR.predict_classes(text_sequence)
prediction_classes = (prediction > 0.5).astype(int)
print(prediction_classes[0][0])

1/1 [==============================] - 0s 29ms/step
1


In [11]:
from tensorflow.keras.optimizers import Adagrad

fakeDETECTOR = Sequential()
fakeDETECTOR.add(Embedding(vocab_size, embedding_dim, input_length=max_sequence_length))
fakeDETECTOR.add(Conv1D(256, 5, activation='relu'))
fakeDETECTOR.add(GlobalMaxPooling1D())
fakeDETECTOR.add(Dense(256, activation='relu'))
fakeDETECTOR.add(Dense(1, activation='sigmoid'))

fakeDETECTOR.compile(optimizer=Adagrad(), loss='binary_crossentropy', metrics=['accuracy'])
fakeDETECTOR.fit(train_sequences, y_train, epochs=4, batch_size=32, validation_data=(val_sequences, y_val))

Epoch 1/4
243/243 [==============================] - 264s 1s/step - loss: 0.6892 - accuracy: 0.5654 - val_loss: 0.6861 - val_accuracy: 0.5710
Epoch 2/4
243/243 [==============================] - 257s 1s/step - loss: 0.6839 - accuracy: 0.5711 - val_loss: 0.6814 - val_accuracy: 0.5710
Epoch 3/4
243/243 [==============================] - 258s 1s/step - loss: 0.6796 - accuracy: 0.5711 - val_loss: 0.6773 - val_accuracy: 0.5710
Epoch 4/4
243/243 [==============================] - 257s 1s/step - loss: 0.6757 - accuracy: 0.5711 - val_loss: 0.6733 - val_accuracy: 0.5710


In [12]:
from tensorflow.keras.optimizers import Adamax

fakeDETECTOR = Sequential()
fakeDETECTOR.add(Embedding(vocab_size, embedding_dim, input_length=max_sequence_length))
fakeDETECTOR.add(Conv1D(256, 5, activation='relu'))
fakeDETECTOR.add(GlobalMaxPooling1D())
fakeDETECTOR.add(Dense(256, activation='relu'))
fakeDETECTOR.add(Dense(1, activation='sigmoid'))

fakeDETECTOR.compile(optimizer=Adamax(), loss='binary_crossentropy', metrics=['accuracy'])
fakeDETECTOR.fit(train_sequences, y_train, epochs=4, batch_size=32, validation_data=(val_sequences, y_val))

Epoch 1/4
243/243 [==============================] - 326s 1s/step - loss: 0.4573 - accuracy: 0.8119 - val_loss: 0.3234 - val_accuracy: 0.8601
Epoch 2/4
243/243 [==============================] - 322s 1s/step - loss: 0.2527 - accuracy: 0.9001 - val_loss: 0.2705 - val_accuracy: 0.8818
Epoch 3/4
243/243 [==============================] - 342s 1s/step - loss: 0.1506 - accuracy: 0.9500 - val_loss: 0.2563 - val_accuracy: 0.8931
Epoch 4/4
243/243 [==============================] - 320s 1s/step - loss: 0.0696 - accuracy: 0.9824 - val_loss: 0.2367 - val_accuracy: 0.9050


In [13]:
from tensorflow.keras.optimizers import Nadam

fakeDETECTOR = Sequential()
fakeDETECTOR.add(Embedding(vocab_size, embedding_dim, input_length=max_sequence_length))
fakeDETECTOR.add(Conv1D(256, 5, activation='relu'))
fakeDETECTOR.add(GlobalMaxPooling1D())
fakeDETECTOR.add(Dense(256, activation='relu'))
fakeDETECTOR.add(Dense(1, activation='sigmoid'))

fakeDETECTOR.compile(optimizer=Nadam(), loss='binary_crossentropy', metrics=['accuracy'])
fakeDETECTOR.fit(train_sequences, y_train, epochs=4, batch_size=32, validation_data=(val_sequences, y_val))

Epoch 1/4
243/243 [==============================] - 330s 1s/step - loss: 0.3534 - accuracy: 0.8445 - val_loss: 0.2381 - val_accuracy: 0.8942
Epoch 2/4
243/243 [==============================] - 329s 1s/step - loss: 0.0829 - accuracy: 0.9721 - val_loss: 0.2365 - val_accuracy: 0.9102
Epoch 3/4
243/243 [==============================] - 325s 1s/step - loss: 0.0110 - accuracy: 0.9965 - val_loss: 0.2912 - val_accuracy: 0.8931
Epoch 4/4
243/243 [==============================] - 324s 1s/step - loss: 0.0074 - accuracy: 0.9964 - val_loss: 0.2946 - val_accuracy: 0.8978


In [15]:
from tensorflow.keras.optimizers import Nadam

fakeDETECTOR = Sequential()
fakeDETECTOR.add(Embedding(vocab_size, embedding_dim, input_length=max_sequence_length))
fakeDETECTOR.add(Conv1D(128, 5, activation='relu'))
fakeDETECTOR.add(GlobalMaxPooling1D())
fakeDETECTOR.add(Dense(128, activation='relu'))
fakeDETECTOR.add(Dense(1, activation='sigmoid'))

fakeDETECTOR.compile(optimizer=Nadam(), loss='binary_crossentropy', metrics=['accuracy'])
fakeDETECTOR.fit(train_sequences, y_train, epochs=2, batch_size=32, validation_data=(val_sequences, y_val))

Epoch 1/2
243/243 [==============================] - 227s 928ms/step - loss: 0.3700 - accuracy: 0.8340 - val_loss: 0.2490 - val_accuracy: 0.8844
Epoch 2/2
243/243 [==============================] - 229s 942ms/step - loss: 0.0970 - accuracy: 0.9675 - val_loss: 0.2334 - val_accuracy: 0.9128
